In [2]:
import pdfplumber
from collections import Counter
import pandas as pd
from tqdm import tqdm
import os

- [adidas](#adidas)
- [Fast Retailing](#FastRetailing)
- [H&M](#H&M)
- [HBI](#HBI)
- [Levis](#Levis)
- [Nike](#Nike)

# adidas:

In [ ]:
adidas2017result=[]
with pdfplumber.open('files/Adidas 2007_November_Factory_List.pdf') as adidas2017:
    for page in adidas2017.pages:
        tables = page.extract_table()
        adidas2017result.extend([x[0].replace('\n',' ') for x in tables if x[0] and x[0].strip() not in ['','Country']])

adidas2017result=Counter(adidas2017result)
# correction:
adidas2017result['China']+=1

In [96]:
adidas=pd.DataFrame(adidas2017result,index=['Adidas 2017']).T.reset_index().rename(columns={'index': 'Country'})

In [97]:
for fileName in tqdm([x for x in os.listdir('files') if x.startswith('Adidas') and '2007' not in x and '2022' not in x]):
    colName=fileName.split('_')[0]
    result=[]
    with pdfplumber.open(f'files/{fileName}') as adidastemp:
        print(f'processing {fileName}')
        for page in adidastemp.pages:
            tables = page.extract_table()
            if not result:
                countryIndex=[x for x in tables if 'Country' in x][0].index('Country')
                print(countryIndex)
            result.extend([x[countryIndex].replace('\n',' ') for x in tables if x[countryIndex] and x[countryIndex].strip() not in ['','Country']])
        result=pd.DataFrame(Counter(result),index=[colName]).T.reset_index().rename(columns={'index': 'Country'})
        adidas=pd.merge(adidas, result, on='Country', how='outer')

  0%|          | 0/3 [00:00<?, ?it/s]

processing Adidas 2009_July_Licensee_Factory_List.pdf
1


 33%|███▎      | 1/3 [00:00<00:01,  1.27it/s]

processing Adidas 2010_Jan_Licensee_Factory_List.pdf
1


 67%|██████▋   | 2/3 [00:01<00:00,  1.22it/s]

processing Adidas 2012_Jan_ Licensee_Factory_List.pdf
1


100%|██████████| 3/3 [00:02<00:00,  1.09it/s]


In [99]:
adidas.to_excel('adidas.xlsx',index=0)

In [11]:
sorted(tables[0], key=lambda x:len(x))

['China',
 'Apache Footwear Ltd.\n(Qingxin)',
 'Trade Union and workers\nrepresentatives are in place.',
 'Industrial Zone, Tai Ping industrial area, Qingxin\nCounty, Qingyuan, Guangdong']

In [13]:
adidas2022result=[]
with pdfplumber.open('files/Adidas 2022_world_cup_supplier_list.pdf') as adidas2022:
    for page in adidas2022.pages:
        tables = page.extract_table()
        adidas2022result.extend([sorted(x, key=lambda x:len(x))[0] for x in tables if x[2] and None not in x and 'Location' not in x])

Counter(adidas2022result)

Counter({'Vietnam': 15,
         'China': 8,
         'Cambodia': 7,
         'Indonesia': 7,
         'Turkey': 4,
         'Pakistan': 2,
         'Thailand': 2,
         'Taiwan': 1,
         'Tunisia': 1,
         'Ukraine': 1,
         'Argentina': 1,
         'Brazil': 1,
         'Colombia': 1,
         'Mexico': 1,
         'Philippines': 1,
         'Italy': 1,
         'Myanmar': 1})

# FastRetailing

In [46]:
fr2023=pd.read_excel('files/Fast Retailing CoreFabricMillList 1March2023.xlsx',names=['.','Country','Factory name','Adress'])
fr2023.drop(fr2023.columns[0],axis=1).iloc[4:]
frResult=pd.DataFrame(fr2023.Country.value_counts()).reset_index().rename({'count':'CoreFabricMillList 1March2023'},axis=1)
frResult

,Country,CoreFabricMillList 1March2023
0,China,72
1,Italy,29
2,Japan,13
3,Vietnam,11
4,Indonesia,9
5,Bangladesh,7
6,Thailand,7
7,India,6
8,South Korea,2
9,France,2


In [48]:
fr2022_3_1=pd.read_excel('files/FastRetailing-Uniqlo-CoreFabricMillList 3-1-2022.xlsx',names=['.','Country','Factory name','Adress'])
fr2022_3_1.drop(fr2022_3_1.columns[0],axis=1).iloc[4:]
frResult=pd.merge(frResult,pd.DataFrame(fr2022_3_1.Country.value_counts()).rename({'count':'Uniqlo-CoreFabricMillList 3-1-2022'},axis=1), on='Country', how='outer')
frResult

,Country,CoreFabricMillList 1March2023,Uniqlo-CoreFabricMillList 3-1-2022
0,Bangladesh,7.0,4.0
1,China,72.0,49.0
2,Country,1.0,1.0
3,France,2.0,2.0
4,India,6.0,1.0
5,Indonesia,9.0,8.0
6,Italy,29.0,1.0
7,Japan,13.0,6.0
8,Korea,NaN,1.0
9,Malaysia,2.0,2.0


In [49]:
for fileName in [x for x in os.listdir('files') if x.startswith('Fast Retailing') and x.endswith('.pdf')]:
    colName=fileName.replace('.pdf','').split("Retailing")[1].strip()
    # print(columnName)
    result=[]
    with pdfplumber.open(f'files/{fileName}') as frTemp:
        print(f'processing {fileName}')
        for page in frTemp.pages:
            tables = page.extract_table()
            if not result:
                countryIndex=[x for x in tables if 'Country' in x][0].index('Country')
                # print(countryIndex)
            result.extend([x[countryIndex].replace('\n',' ') for x in tables if x[countryIndex] and x[countryIndex].strip() not in ['','Country']])
        result=pd.DataFrame(Counter(result),index=[colName]).T.reset_index().rename(columns={'index': 'Country'})
        frResult=pd.merge(frResult, result, on='Country', how='outer')

processing Fast Retailing CoreFabricMillList 1Sept2022.pdf
processing Fast Retailing CoreSewingFactoryList March 2021.pdf
processing Fast Retailing CoreSewingFactoryList_20190329.pdf
processing Fast Retailing CoreSewingFactoryList_20200329.pdf
processing Fast Retailing GarmentProcessingFtyList 1Sept2022.pdf
processing Fast Retailing GarmentProcessingFtyList March 2022.pdf
processing Fast Retailing GU Core Partner Factory List_20171215.pdf
processing Fast Retailing GUCorePartnerFactoryList_20171215.pdf
processing Fast Retailing UniqloCorePartnerFactoryList_20180330.pdf


In [50]:
frResult

,Country,CoreFabricMillList 1March2023,Uniqlo-CoreFabricMillList 3-1-2022,CoreFabricMillList 1Sept2022,CoreSewingFactoryList March 2021,CoreSewingFactoryList_20190329,CoreSewingFactoryList_20200329,GarmentProcessingFtyList 1Sept2022,GarmentProcessingFtyList March 2022,GU Core Partner Factory List_20171215,GUCorePartnerFactoryList_20171215,UniqloCorePartnerFactoryList_20180330
0,Bangladesh,7.0,4.0,4.0,23.0,24.0,24.0,41.0,41.0,13.0,13.0,10.0
1,Bulgaria,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cambodia,NaN,NaN,NaN,16.0,10.0,9.0,28.0,28.0,7.0,7.0,5.0
3,China,72.0,49.0,47.0,167.0,128.0,134.0,302.0,312.0,26.0,26.0,110.0
4,Country,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,France,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,India,6.0,1.0,1.0,9.0,6.0,8.0,14.0,17.0,NaN,NaN,NaN
7,Indonesia,9.0,8.0,7.0,19.0,17.0,14.0,23.0,23.0,5.0,5.0,16.0
8,Italy,29.0,1.0,1.0,2.0,NaN,NaN,6.0,6.0,NaN,NaN,NaN
9,Japan,13.0,6.0,6.0,1.0,2.0,2.0,19.0,22.0,NaN,NaN,2.0


In [51]:
frResult.to_csv('FastRetailing.csv',index=0)

# H&M

In [7]:
hm=pd.read_csv('files/H&M 23March2020 supplier list - downloaded via OAR.csv')
hmresult=pd.DataFrame(hm.country_name.value_counts()).reset_index()

# HBI

In [74]:
hbiResult=None
flag=0
for fileName in [x for x in os.listdir('files') if x.startswith('HBI') and x.endswith('.pdf')]:
    colName=fileName.replace('.pdf','').split("#")[1].strip()
    result=[]
    with pdfplumber.open(f'files/{fileName}') as hbiTemp:
        print(f'processing {fileName}')
        for page in hbiTemp.pages:
            tables = page.extract_table()
            if not result:
                for i in tables:
                    if None in i:
                        continue
                    else:
                        countryIndex=[ii for ii in range(len(i)) if 'country' in i[ii].lower()][0]
                        break                
            result.extend([i[countryIndex] for i in tables if None not in i and 'country' not in i[countryIndex].lower()])
    
    result=pd.DataFrame(Counter(result),index=[colName]).T.reset_index().rename({'index':'Country'},axis=1)
    if flag==0:
        hbiResult=result
        flag=1
    else:
        hbiResult=pd.merge(hbiResult,result, on='Country', how='outer')
    # hbiResult.extend(result)

processing HBI#Fty-List-Website-Owned Hanesbrands Oct2022.pdf
processing HBI#Owned Hanes 2023-Feb-Factory-list.pdf
processing HBI#Self-Owned 2017.pdf
processing HBI#Self-Owned 2022-4-11.pdf
processing HBI#Self-Owned-122020.pdf


In [76]:
hbiResult.to_csv('HBI.csv',index=0)

# Levis


In [9]:
def levisHelper(inp):
    result={}
    last=None
    for i in inp:
        if i is not None:
            result[i]=0
            last=i
        else:
            result[last]+=1
    return result

In [30]:
levisResult=None
flag=0
for i in [x for x in os.listdir('files') if x.startswith('Levi') and x.endswith('.pdf')]:
    print(f'processing {i}...')
    colName=i.replace('.pdf','').split(' ')[1]
    with pdfplumber.open(f'files/{i}') as levisTemp:
        result=[]
        for page in levisTemp.pages:
            tables = page.extract_table()
            result.extend([x[0] for x in tables if x[0] not in ['Country' , 'country','']])
        result=pd.DataFrame(levisHelper(result),index=[colName]).T.reset_index().rename({'index':'Country'},axis=1)
        if flag==0:
            levisResult=result
            flag=1
        else:
            levisResult=pd.merge(levisResult,result, on='Country', how='outer')
    
levisResult

processing Levi-Strauss-Co-Factory-List December-2014-Revised.pdf...
processing Levi-Strauss-Co-Factory-List December-2015.pdf...
processing Levi-Strauss-Co-Factory-List June-2015.pdf...
processing Levi-Strauss-Co-Factory-List September-2014.pdf...


,Country,December-2014-Revised,December-2015,June-2015,September-2014
0,Argentina,27.0,24.0,26.0,30.0
1,Bangladesh,10.0,15.0,10.0,10.0
2,Brazil,1.0,1.0,1.0,NaN
3,Bulgaria,5.0,4.0,4.0,5.0
4,Cambodia,13.0,11.0,13.0,13.0
5,China,191.0,170.0,194.0,194.0
6,Colombia,3.0,3.0,3.0,4.0
7,Dominican\nRepublic,8.0,NaN,NaN,9.0
8,Dominican Republic,NaN,8.0,8.0,NaN
9,Egypt,8.0,11.0,10.0,8.0


In [31]:
levisResult.to_csv('levis.csv',index=0)

# Nike

file corrupted, unable to read

In [41]:
# with pdfplumber.open(f'files/NIKE Dec09_Collegiate_Disclosure.pdf', ) as nikeTemp:
#         # result=[]
#         for page in nikeTemp.pages:
#             tables = page.extract_table()
#             print(tables[0])
nike2019={
    'Bangladesh':2,
    'Cambodia':2,
    'Canada':1,
    'China':30,
    'El Salvador':1,
    'Honduras':2,
    'Hong Kong':2,
    'Indonesia':14,
    'Isarel':1,
    'Korea':1,
    'Malaysia':1,
    'Mexico':6,
    'Taiwan':11,
    'Thailand':9,
    'USA':36,
    'Vietnam':19
}

nikeResult=pd.DataFrame(nike2019,index=['2009']).T.reset_index().rename({'index':'Country'},axis=1)

In [50]:
nike2012=pd.read_csv('files/Nike Jan 2012 w Umbro, Converse and Hurley.csv', header=1)
nike2012=pd.DataFrame(nike2012.Country.value_counts()).reset_index().rename({'count':'2012 All'},axis=1)
nikeResult=pd.merge(nikeResult, nike2012, on='Country', how='outer')
nikeResult.to_csv('nike.csv', index=0)

# Uniqlo

In [58]:
with pdfplumber.open(f'files/UniqloCoreFabricMillList_2020Aug.pdf', ) as uniqloTemp:
        result=[]
        for page in uniqloTemp.pages:
            tables = page.extract_table()
            result.extend([x[1] for x in tables])

dict(Counter(result))

{'Country': 1,
 'Bangladesh': 1,
 'China': 43,
 'Indonesia': 5,
 'Japan': 6,
 'Korea': 1,
 'Malaysia': 1,
 'Thailand': 5,
 'Turkey': 1,
 'Vietnam': 9}